In [ ]:
import PyDSTool as dst
from PyDSTool import args
from PyDSTool.Toolbox import phaseplane as pp

import numpy as np
from matplotlib import pyplot as plt

In [ ]:
pars = {'h': .005, 'A': 7.26722e27,'n': 3.5,'Q': 52.,'iPe': 1000, 'Di': 2.}
edot0 = 1.e-11

pars['A']=6500/edot0*5e4**3.5*1e4**-2
pars['iPe'] = 7.38425825
icdict = {'T': 1.2,'tau': 0.24}
Tstr = '(iPe)*(1.-T)+Di*h*(A*tau**n)*exp(-Q/T)'
taustr = '2*(1-h*(A*tau**n)*exp(-Q/T))'
print pars['A']

$$\vec{X}= \begin{pmatrix}  T \\ 
\tau \end{pmatrix} $$

$$\vec{X}'= \underline{F}(\vec{X})$$

$$\underline{F}(\vec{X})=\begin{pmatrix} \frac{1}{Pe}(1-T)+Di(h)(A\tau^{n}e^{-Q/T}) \\
 2(1-h(A\tau^{n}e^{-Q/T}))\end{pmatrix}$$
the jacobian:
$$\underline{J}(\vec{X})=\begin{pmatrix}  \frac{\partial F_T}{\partial T} & \frac{\partial F_T}{\partial \tau}\\
         \frac{\partial F_\tau}{\partial T} & \frac{\partial F_\tau}{\partial\tau} \end{pmatrix}$$

In [ ]:

DSargs = args(name='T v tau')  # struct-like data
#DSargs.events = [event_T_A]
DSargs.pars = pars
DSargs.tdata = [0, 3.]
DSargs.algparams = {'max_pts': 3000, 'init_step': 0.02, 'stiff': True}
DSargs.varspecs = {'T': Tstr, 'tau': taustr}
DSargs.xdomain = {'T': [1.,3.], 'tau': [0., 3.]}
DSargs.fnspecs = {'Jacobian': (['t','T','tau'],
                                """[[-iPe + (Di)*(h)*(A)*(tau**n)*(Q/T/T)*(exp(-Q/T)), (Di)*(h)*(A)*exp(-Q/T)*(n)*tau**(n-1)],
                                    [-2*(h)*(A)*(tau**n)*(Q/T/T)*exp(-Q/T)  ,  -2*(A)*exp(-Q/T)*(h)*(n)*tau**(n-1)]]""")}
DSargs.ics = icdict
vdp = dst.Vode_ODEsystem(DSargs)


In [ ]:
N = 30
iPeRange = np.linspace(5.,20.,N)
fp_T = np.zeros(N)
fp_tau = np.zeros(N)
fp_det = np.zeros(N)
fp_trace = np.zeros(N)
t = 0.

for i,iPe in enumerate(iPeRange):
    vdp.pars['iPe']=iPe
    try:
        fp_coords = pp.find_fixedpoints(vdp, n=10, eps=1e-8)
        fp_coord = fp_coords[0]
        fp_T[i] = fp_coord['T']
        fp_tau[i] = fp_coord['tau']
        eig = np.linalg.eig(vdp.Jacobian(t,fp_coord))[0]
        fp_det[i] = eig[0]*eig[1]
        fp_trace[i] = np.sum(eig)
        if len(fp_coords) > 1:
            print('Warning: {} fixpoints'.format(len(fp_coords)))
            print(fp_coords)
    except:
        print(" some dumb error")
        
    
    
    print('iPe = {}:\n fixed-point: T= {}, tau={}, Tr = {}, eigs={}'.format(iPe,fp_T[i],fp_tau[i],fp_trace[i],eig))  
plt.figure(1)
plt.plot(iPeRange, fp_T, 'b', linewidth=1.7,label='T')
plt.plot(iPeRange, fp_tau, 'r', linewidth=1.7,label='$\\tau$')
plt.legend(loc='best')
plt.show()

plt.figure(2)
plt.plot(iPeRange, fp_det, 'g', linewidth=1.,label='$|J|$')
plt.plot(iPeRange, fp_trace, 'y', linewidth=1.,label='$Tr(J)$')
plt.plot(iPeRange, np.zeros(iPeRange.shape), 'k:')
plt.legend(loc='best')
plt.show()

In [ ]:
Tr=np.linspace(-100,100)
DetCr = Tr**2/4.
plt.figure(3)
plt.scatter(fp_trace, fp_det, s=40, c=iPeRange, cmap='Set3')
plt.plot(Tr,DetCr,'k:')
plt.grid()
#marker=None, cmap=None, norm=None, vmin=None, vmax=None, alpha=None, linewidths=None, verts=None, edgecolors=None, **kwargs

plt.colorbar()
plt.xlim(-100,100)
plt.show()

#trace=0 at around iPe=7.38425825

In [ ]:
pars['iPe'] = 7.38425825
N = 30
DiRange = np.linspace(0.,10.,N)
fp_T = np.zeros(N)
fp_tau = np.zeros(N)
fp_det = np.zeros(N)
fp_trace = np.zeros(N)
t = 0.

for i,Di in enumerate(DiRange):
    vdp.pars['Di']=Di
    try:
        fp_coords = pp.find_fixedpoints(vdp, n=10, eps=1e-8)
        fp_coord = fp_coords[0]
        fp_T[i] = fp_coord['T']
        fp_tau[i] = fp_coord['tau']
        eig = np.linalg.eig(vdp.Jacobian(t,fp_coord))[0]
        fp_det[i] = eig[0]*eig[1]
        fp_trace[i] = np.sum(eig)
        if len(fp_coords) > 1:
            print('Warning: {} fixpoints'.format(len(fp_coords)))
            print(fp_coords)
    except:
        print(" some dumb error")
        
    
    
    print('Di = {}:\n fixed-point: T= {}, tau={}, Tr = {}, eigs={}'.format(Di,fp_T[i],fp_tau[i],fp_trace[i],eig))  
plt.figure(4)
plt.plot(DiRange, fp_T, 'b', linewidth=1.7,label='T')
plt.plot(DiRange, fp_tau, 'r', linewidth=1.7,label='$\\tau$')
plt.legend(loc='best')
plt.show()

plt.figure(5)
plt.plot(DiRange, fp_det, 'g', linewidth=1.,label='$|J|$')
plt.plot(DiRange, fp_trace, 'y', linewidth=1.,label='$Tr(J)$')
plt.plot(DiRange, np.zeros(iPeRange.shape), 'k:')
plt.legend(loc='best')
plt.show()


In [ ]:
Tr=np.linspace(-10,10)
DetCr = Tr**2/4.
plt.figure(6)
plt.scatter(fp_trace, fp_det, s=40, c=DiRange)
plt.plot(Tr,DetCr,'k:')
plt.grid()


plt.colorbar()
plt.xlim(-100,100)
plt.show()
plt.show()